In [8]:
from typing import Tuple, List
import nibabel as nib
import json
import gdown
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual


import sys
sys.path.append('..')

from markers.utils import * 
from markers.morphology import erode_segmentation, compute_suv

import torch
USE_GPU = torch.cuda.is_available()
print(USE_GPU)

os.getcwd()

False


'/Users/markus/GitHub/PACKAGES/markers/examples'

# Get Data

In [4]:
DUMMY_DATA = os.path.join('..', 'data')
with open(os.path.join(DUMMY_DATA, 'gdrive.json')) as f:
    data = json.load(f)


 
for filename, gdrive_link in data.items():
    if os.path.exists(os.path.join(DUMMY_DATA, filename)):
        print(f'{filename} already exists, skipping...')
        continue

    # extract the file id from the link
    file_id = gdrive_link.split('/')[-2]

    # build download link
    download_link = f'https://drive.google.com/uc?id={file_id}'
    try:
        gdown.download(download_link, os.path.join(DUMMY_DATA, filename), quiet=False)
    except Exception as e:
        print(f'Error with {filename}:\n{e}')

CT.nii.gz already exists, skipping...
segmentation.nii.gz already exists, skipping...


Downloading...
From: https://drive.google.com/uc?id=1yBeopAALgt8BhEiH8d11JUj4C6UNuhW2
To: /Users/markus/GitHub/PACKAGES/markers/data/highlighted_resampled.nii.gz
100%|██████████| 57.3k/57.3k [00:00<00:00, 611kB/s]
Downloading...
From: https://drive.google.com/uc?id=1yBeopAALgt8BhEiH8d11JUj4C6UNuhW2
To: /Users/markus/GitHub/PACKAGES/markers/data/resampled_segmentation.nii.gz
100%|██████████| 57.3k/57.3k [00:00<00:00, 673kB/s]
Downloading...
From: https://drive.google.com/uc?id=12qa8qFkrO9-alMAVj-uqyn4B4CgzX_kc
To: /Users/markus/GitHub/PACKAGES/markers/data/resampled_CT.nii.gz
100%|██████████| 12.3M/12.3M [00:02<00:00, 5.03MB/s]


# Create Segmentations

In [6]:
from markers.create_segs import create_segs

create_segs(input_path=os.path.join(DUMMY_DATA, 'CT.nii.gz'),
            output_path=os.path.join(DUMMY_DATA, 'segs'),
            task='total',
            body_seg=True,
            roi_subset=None, # use default segmentations
            )
create_segs(input_path=os.path.join(DUMMY_DATA, 'CT.nii.gz'),
            output_path=os.path.join(DUMMY_DATA, 'segs'),
            task='heartchambers_highres',
            body_seg=True,
            roi_subset=None, # will be ignored anyways
            )


If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

No GPU detected. Running on CPU. This can be very slow. The '--fast' or the `--roi_subset` option can help to reduce runtime.
Generating rough body segmentation...
Resampling...
  Resampled in 5.79s
Predicting...


/Users/markus/GitHub/LOCAL/lib/python3.11/site-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "
100%|██████████| 28/28 [00:34<00:00,  1.22s/it]


  Predicted in 41.45s
Resampling...
  cropping from (512, 512, 541) to (178, 185, 59)
Predicting...


100%|██████████| 12/12 [09:25<00:00, 47.16s/it]


  Predicted in 578.06s
Saving segmentations...


  0%|          | 0/7 [00:00<?, ?it/s]

Creating heart_myocardium.nii.gzCreating heart_atrium_left.nii.gz

Creating heart_ventricle_left.nii.gz
Creating heart_atrium_right.nii.gz
Creating heart_ventricle_right.nii.gz
Creating aorta.nii.gz
Creating pulmonary_artery.nii.gz
  Saved in 9.02s


# Compute SUV


In [ ]:
from markers.suv import SUV

instance = SUV(segs_path=DUMMY_DATA,
               spect_path=os.path.join(DUMMY_DATA, 'SPECT.nii.gz'),
               threshold=0.4,
               mm_to_erode=3,
               suv_cube_vol=4,
               suv_method='peak',
               use_convolution=True,
               use_gpu=USE_GPU,
               verbose=True)

rtidx = instance.compute_retention_idx(vertebrae="T9", use_median_vertebrae=False)
suv_peak = instance.compute_suv(body_part='heart_myocardium', preprocess=True)
instance._free_memory(verbose=True)
print(rtidx, suv_peak)

# Compute TBR

In [ ]:
from markers.tbr import TBR

instance = TBR(segs_path=DUMMY_DATA,
               spect_path=os.path.join(DUMMY_DATA, 'SPECT.nii.gz'),
               threshold=0.4,
               mm_to_erode=3,
               tbr_cube_vol=4,
               tbr_method='max',
               use_convolution=True,
               use_gpu=USE_GPU,
               verbose=True)

tbr_peak = instance.compute_tbr(body_part='heart_myocardium', preprocess=True)
instance._free_memory(verbose=True)
print(tbr_peak)

# Compute CT Segmentation Volume

In [ ]:
from markers.volume import Volume

instance = Volume(segs_path=DUMMY_DATA,
                  segs_subset='heart_myocardium',
                  mm_to_erode=0
                  )

vol = instance.compute_volume(compute_number=True)
instance.compute_volume(compute_number=False)
instance.plot_heatmaps()
instance._free_memory(verbose=True)
print(vol)

# Compute SPECT Uptake Volume using SPECT Segmentation Masks

In [ ]:
from markers.uptake import UptakeVol

instance = UptakeVol(spect_path=os.path.join(DUMMY_DATA, 'SPECT.nii.gz'),
                     segs_path=os.path.join(DUMMY_DATA, 'segs'),
                     segs_subset='heart_myocardium',
                     mm_to_dilate=10,
                     approach='threshold-bb',
                     threshold=0.4)
uptake_vol = instance.compute_uptake_vol(compute_number=True, verbose=True)
instance.compute_uptake_vol(compute_number=False, verbose=True)
instance.plot_heatmaps()
instance._free_memory(verbose=True)
print(uptake_vol)


# Compute Interventricular Septum Volume

In [ ]:
from markers.septum import SeptumVol

instance = SeptumVol(segs_path=os.path.join(DUMMY_DATA, 'segs'),
                     seg_subset_names=['heart_ventricle_left', 'heart_ventricle_right'],
                     mm_to_dilate=10,
                     verbose=True)

septum_vol = instance.compute_septum_vol(compute_number=True, verbose=True)
instance.compute_septum_vol(compute_number=False, verbose=True)
instance.vizz_septum()
instance._free_memory(verbose=True)
print(septum_vol)